# Testing power of agent architectures for graph isomorphism

## Setup

In [67]:
FORCE_CPU = True

SAVE_DATA = False
LOAD_DATA = True

SEED = 349287

DATASET_NAME = "er10000"
TEST_SIZE = 0.2

D_DECIDER = 16

FREEZE_ENCODER = False

BATCH_SIZE = 256
NUM_EPOCHS = 500
LEARNING_RATE = 0.003
SCHEDULER_PATIENCE = 2000
SCHEDULER_FACTOR = 0.5

freeze_text = "_freeze" if FREEZE_ENCODER else ""
RESULTS_FILE = f"data/gi_agent_test_results_{DATASET_NAME}{freeze_text}.pkl"

In [68]:
from abc import ABC
import pickle

import numpy as np

import torch
from torch import nn
import torch.nn.functional as F
from torch import Tensor
from torch.utils.data import random_split
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau

from torch_geometric.data import Batch as GeometricBatch
from torch_geometric.loader import DataLoader
from torch_geometric.transforms import BaseTransform

from jaxtyping import Float

from tqdm import tqdm

import plotly.graph_objs as go

from pvg.scenarios import GraphIsomorphismAgent
from pvg.data import GraphIsomorphismDataset, GraphIsomorphismData
from pvg.parameters import Parameters, GraphIsomorphismParameters

In [69]:
torch.manual_seed(SEED)
np.random.seed(SEED)
torch_generator = torch.Generator().manual_seed(SEED)

In [70]:
params = Parameters(
    scenario="graph_isomorphism",
    trainer="test",
    dataset=DATASET_NAME,
    max_message_rounds=1,
    graph_isomorphism=GraphIsomorphismParameters(
        prover_d_gnn=8,
        verifier_d_gnn=8,
    )
)

In [71]:
if not FORCE_CPU and torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(device)

cpu


## Load dataset

In [72]:
class ScoreToBitTransform(BaseTransform):
    """A transform that converts the score to a bit indicating isomorphism."""
    def __call__(self, data):
        for store in data.node_stores:
            store.y = (store.wl_score == -1).long()
        return data

In [73]:
dataset = GraphIsomorphismDataset(params, transform=ScoreToBitTransform())
dataset

GraphIsomorphismDataset(name='er10000', num_features=1, num_pairs=10000)

In [74]:
train_dataset, test_dataset = random_split(dataset, (1 - TEST_SIZE, TEST_SIZE))

## Agents

In [75]:
class GraphIsomorphismSoloAgent(GraphIsomorphismAgent, ABC):
    def _build_model(
        self, num_layers: int, d_gnn: int, d_gin_mlp: int, num_heads: int
    ) -> nn.Module:
        # Build up the GNN module
        self.gnn, self.attention = self._build_gnn_and_attention(
            d_input=1,
            d_gnn=d_gnn,
            d_gin_mlp=d_gin_mlp,
            num_layers=num_layers,
            num_heads=num_heads,
        )

        # Build the decider, which decides whether the graphs are isomorphic
        self.decider = self._build_decider(
            d_gnn=d_gnn,
            d_decider=D_DECIDER,
            d_out=2,
        )

    def forward(
        self, data: GraphIsomorphismData | GeometricBatch
    ) -> Float[Tensor, "batch_size 2"]:
        gnn_output, attention_output, _ = self._run_gnn_and_attention(data)
        gnn_attn_output = gnn_output + attention_output
        decider_logits = self.decider(gnn_output)
        return decider_logits

    def to(self, device: str | torch.device):
        self.gnn.to(device)
        self.attention.to(device)
        self.decider.to(device)
        return self


class GraphIsomorphismSoloProver(GraphIsomorphismSoloAgent):
    def __init__(self, params: Parameters, device: str | torch.device):
        super().__init__(params, device)
        self._build_model(
            num_layers=params.graph_isomorphism.prover_num_layers,
            d_gnn=params.graph_isomorphism.prover_d_gnn,
            d_gin_mlp=params.graph_isomorphism.prover_d_gin_mlp,
            num_heads=params.graph_isomorphism.prover_num_heads,
        )


class GraphIsomorphismSoloVerifier(GraphIsomorphismSoloAgent):
    def __init__(self, params: Parameters, device: str | torch.device):
        super().__init__(params, device)
        self._build_model(
            num_layers=params.graph_isomorphism.verifier_num_layers,
            d_gnn=params.graph_isomorphism.verifier_d_gnn,
            d_gin_mlp=params.graph_isomorphism.verifier_d_gin_mlp,
            num_heads=params.graph_isomorphism.verifier_num_heads,
        )

In [76]:
prover = GraphIsomorphismSoloProver(params, device)
prover

GraphIsomorphismSoloProver(
  (gnn): Sequential(
    (0): Linear(1, 8, bias=True)
    (1): ReLU(inplace=True)
    (2): GINConv(nn=Sequential(
    (0): Linear(in_features=8, out_features=64, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=64, out_features=8, bias=True)
  ))
    (3): ReLU(inplace=True)
    (4): GINConv(nn=Sequential(
    (0): Linear(in_features=8, out_features=64, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=64, out_features=8, bias=True)
  ))
    (5): ReLU(inplace=True)
    (6): GINConv(nn=Sequential(
    (0): Linear(in_features=8, out_features=64, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=64, out_features=8, bias=True)
  ))
    (7): ReLU(inplace=True)
    (8): GINConv(nn=Sequential(
    (0): Linear(in_features=8, out_features=64, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=64, out_features=8, bias=True)
  ))
    (9): ReLU(inplace=True)
    (10): GINConv(nn=Sequential(
    (0): Line

In [77]:
verifier = GraphIsomorphismSoloVerifier(params, device)
verifier

GraphIsomorphismSoloVerifier(
  (gnn): Sequential(
    (0): Linear(1, 8, bias=True)
    (1): ReLU(inplace=True)
    (2): GINConv(nn=Sequential(
    (0): Linear(in_features=8, out_features=64, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=64, out_features=8, bias=True)
  ))
    (3): ReLU(inplace=True)
    (4): GINConv(nn=Sequential(
    (0): Linear(in_features=8, out_features=64, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=64, out_features=8, bias=True)
  ))
  )
  (attention): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=8, out_features=8, bias=True)
  )
  (decider): Sequential(
    (0): ReLU(inplace=True)
    (1): Linear(in_features=8, out_features=16, bias=True)
    (2): ReLU(inplace=True)
    (3): Reduce('pair batch_size max_nodes d_decider -> pair batch_size d_decider', 'max')
    (4): Rearrange('pair batch_size d_decider -> batch_size (pair d_decider)')
    (5): Linear(in_features=32, out_features=16, bi

## Train

In [78]:
if FREEZE_ENCODER:
    prover_train_params = []
    for name, param in prover.named_parameters():
        if name.startswith("gnn") or name.startswith("attention"):
            param.requires_grad = False
        else:
            prover_train_params.append(param)
    verifier_train_params = []
    for name, param in verifier.named_parameters():
        if name.startswith("gnn") or name.startswith("attention"):
            param.requires_grad = False
        else:
            verifier_train_params.append(param)
else:
    prover_train_params = prover.parameters()
    verifier_train_params = verifier.parameters()

[autoreload of Sequential_d08248 failed: Traceback (most recent call last):
  File "/home/sam/.virtualenvs/pvg-experiments/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/home/sam/.virtualenvs/pvg-experiments/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/importlib/__init__.py", line 168, in reload
    raise ModuleNotFoundError(f"spec not found for the module {name!r}", name=name)
ModuleNotFoundError: spec not found for the module 'Sequential_d08248'
]


In [79]:
if not LOAD_DATA:
    optimizer_prover = Adam(prover_train_params, lr=LEARNING_RATE)
    optimizer_verifier = Adam(verifier_train_params, lr=LEARNING_RATE)

    scheduler_prover = ReduceLROnPlateau(
        optimizer_prover,
        "min",
        patience=SCHEDULER_PATIENCE,
        factor=SCHEDULER_FACTOR,
        verbose=True,
    )
    scheduler_verifier = ReduceLROnPlateau(
        optimizer_verifier,
        "min",
        patience=SCHEDULER_PATIENCE,
        factor=SCHEDULER_FACTOR,
        verbose=True,
    )

    test_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        generator=torch_generator,
        follow_batch=["x_a", "x_b"],
    )

    def train_step(
        model: GraphIsomorphismSoloAgent,
        optimizer,
        scheduler,
        data: GraphIsomorphismData,
    ) -> tuple[float, float]:
        model.train()
        optimizer.zero_grad()
        pred = model(data)
        loss = F.cross_entropy(pred, data.y)
        loss.backward()
        optimizer.step()
        scheduler.step(loss)
        with torch.no_grad():
            accuracy = (pred.argmax(dim=1) == data.y).float().mean().item()
        return loss.item(), accuracy

    prover.to(device)
    verifier.to(device)

    losses_prover = np.empty(NUM_EPOCHS)
    accuracies_prover = np.empty(NUM_EPOCHS)
    losses_verifier = np.empty(NUM_EPOCHS)
    accuracies_verifier = np.empty(NUM_EPOCHS)
    for epoch in tqdm(range(NUM_EPOCHS), desc=f"Training"):
        total_loss_prover = 0
        total_accuracy_prover = 0
        total_loss_verifier = 0
        total_accuracy_verifier = 0
        for data in test_loader:
            data = data.to(device)
            loss, accuracy = train_step(
                prover, optimizer_prover, scheduler_prover, data
            )
            total_loss_prover += loss
            total_accuracy_prover += accuracy
            loss, accuracy = train_step(
                verifier, optimizer_verifier, scheduler_verifier, data
            )
            total_loss_verifier += loss
            total_accuracy_verifier += accuracy
        losses_prover[epoch] = total_loss_prover / len(test_loader)
        accuracies_prover[epoch] = total_accuracy_prover / len(test_loader)
        losses_verifier[epoch] = total_loss_verifier / len(test_loader)
        accuracies_verifier[epoch] = total_accuracy_verifier / len(test_loader)
        # print(
        #     f"Prover: loss: {losses_prover[epoch]:.4f}, "
        #     f"accuracy: {accuracies_prover[epoch]:.4%}"
        # )
        # print(
        #     f"Verifier: loss: {losses_verifier[epoch]:.4f}, "
        #     f"accuracy: {accuracies_verifier[epoch]:.4%}"
        # )

## Test

In [80]:
if not LOAD_DATA:
    test_loader = DataLoader(
        test_dataset, batch_size=BATCH_SIZE, shuffle=False, follow_batch=["x_a", "x_b"]
    )


    def test_step(
        model: GraphIsomorphismSoloAgent, optimizer, data: GraphIsomorphismData
    ) -> tuple[float, float]:
        model.eval()
        with torch.no_grad():
            pred = model(data)
            loss = F.cross_entropy(pred, data.y)
            accuracy = (pred.argmax(dim=1) == data.y).float().mean().item()
        return loss.item(), accuracy

    test_loss_prover = 0
    test_accuracy_prover = 0
    test_loss_verifier = 0
    test_accuracy_verifier = 0
    for data in tqdm(test_loader, desc=f"Testing..."):
        data = data.to(device)
        loss, accuracy = test_step(prover, optimizer_prover, data)
        test_loss_prover += loss
        test_accuracy_prover += accuracy
        loss, accuracy = test_step(verifier, optimizer_verifier, data)
        test_loss_verifier += loss
        test_accuracy_verifier += accuracy
    test_loss_prover = test_loss_prover / len(test_loader)
    test_accuracy_prover = test_accuracy_prover / len(test_loader)
    test_loss_verifier = test_loss_verifier / len(test_loader)
    test_accuracy_verifier = test_accuracy_verifier / len(test_loader)

## Saving

In [81]:
if SAVE_DATA:
    with open(RESULTS_FILE, "wb") as f:
        pickle.dump(
            {
                "params": params,
                "losses_prover": losses_prover,
                "accuracies_prover": accuracies_prover,
                "losses_verifier": losses_verifier,
                "accuracies_verifier": accuracies_verifier,
                "test_loss_prover": test_loss_prover,
                "test_accuracy_prover": test_accuracy_prover,
                "test_loss_verifier": test_loss_verifier,
                "test_accuracy_verifier": test_accuracy_verifier,
            },
            f,
        )

## Results

In [82]:
if LOAD_DATA:
    with open(RESULTS_FILE, "rb") as f:
        data = pickle.load(f)
        params = data["params"]
        losses_prover = data["losses_prover"]
        accuracies_prover = data["accuracies_prover"]
        losses_verifier = data["losses_verifier"]
        accuracies_verifier = data["accuracies_verifier"]
        test_loss_prover = data["test_loss_prover"]
        test_accuracy_prover = data["test_accuracy_prover"]
        test_loss_verifier = data["test_loss_verifier"]
        test_accuracy_verifier = data["test_accuracy_verifier"]

In [83]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=np.arange(NUM_EPOCHS), y=losses_prover, mode="lines", name="Prover Loss"
    )
)
fig.add_trace(
    go.Scatter(
        x=np.arange(NUM_EPOCHS), y=losses_verifier, mode="lines", name="Verifier Loss"
    )
)

fig.update_layout(
    title="Prover and Verifier Train Losses", xaxis_title="Epoch", yaxis_title="Loss"
)

fig.show()

In [84]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=np.arange(NUM_EPOCHS),
        y=accuracies_prover,
        mode="lines",
        name="Prover Accuracy",
    )
)
fig.add_trace(
    go.Scatter(
        x=np.arange(NUM_EPOCHS),
        y=accuracies_verifier,
        mode="lines",
        name="Verifier Accuracy",
    )
)

fig.update_layout(
    title="Prover and Verifier Train Accuracies",
    xaxis_title="Epoch",
    yaxis_title="Accuracy",
    yaxis_tickformat=",.0%",
)

fig.show()

In [85]:
print()
print("Final Results:")
print(
    f"Prover: loss: {test_loss_prover:.4f}, "
    f"accuracy: {test_accuracy_prover:.4%}"
)
print(
    f"Verifier: loss: {test_loss_verifier:.4f}, "
    f"accuracy: {test_accuracy_verifier:.4%}"
)


Final Results:
Prover: loss: 0.5594, accuracy: 66.2973%
Verifier: loss: 0.6932, accuracy: 49.5605%


# Testing the prover's decider

In [86]:
decider = GraphIsomorphismSoloProver(params, device).decider
decider

Sequential(
  (0): ReLU(inplace=True)
  (1): Linear(in_features=8, out_features=16, bias=True)
  (2): ReLU(inplace=True)
  (3): Reduce('pair batch_size max_nodes d_decider -> pair batch_size d_decider', 'max')
  (4): Rearrange('pair batch_size d_decider -> batch_size (pair d_decider)')
  (5): Linear(in_features=32, out_features=16, bias=True)
  (6): ReLU(inplace=True)
  (7): Linear(in_features=16, out_features=16, bias=True)
  (8): ReLU(inplace=True)
  (9): Linear(in_features=16, out_features=2, bias=True)
)

In [87]:
d_gnn = params.graph_isomorphism.prover_d_gnn
max_nodes = 10

[autoreload of Sequential_d08248 failed: Traceback (most recent call last):
  File "/home/sam/.virtualenvs/pvg-experiments/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/home/sam/.virtualenvs/pvg-experiments/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/importlib/__init__.py", line 168, in reload
    raise ModuleNotFoundError(f"spec not found for the module {name!r}", name=name)
ModuleNotFoundError: spec not found for the module 'Sequential_d08248'
]


In [88]:
def create_data():
    x_non_iso = torch.randn((2, BATCH_SIZE, max_nodes, d_gnn), device=device)
    x_iso_0 = torch.randn((1, BATCH_SIZE, max_nodes, d_gnn), device=device)
    x_iso_1 = x_iso_0[:, :, torch.randperm(max_nodes), :]
    x_iso = torch.cat((x_iso_0, x_iso_1), dim=0)
    x = torch.cat((x_non_iso, x_iso), dim=1)
    y = torch.cat(
        (
            torch.zeros(BATCH_SIZE, device=device),
            torch.ones(BATCH_SIZE, device=device),
        )
    ).long()
    perm = torch.randperm(2 * BATCH_SIZE)
    x = x[:, perm, :, :]
    y = y[perm]
    return x, y

In [89]:
optimizer = Adam(decider.parameters(), lr=LEARNING_RATE)
for epoch in tqdm(range(1000)):
    optimizer.zero_grad()
    x, y = create_data()
    pred = decider(x)
    loss = F.cross_entropy(pred, y)
    loss.backward()
    optimizer.step()

100%|██████████| 1000/1000 [00:03<00:00, 321.74it/s]


In [90]:
x, y = create_data()
with torch.no_grad():
    pred = decider(x)
    accuracy = (pred.argmax(dim=1) == y).float().mean().item()
print(f"Accuracy: {accuracy:.4%}")

Accuracy: 99.8047%


# Looking at encoded graphs

In [91]:
prover = GraphIsomorphismSoloProver(params, device)
prover

GraphIsomorphismSoloProver(
  (gnn): Sequential(
    (0): Linear(1, 8, bias=True)
    (1): ReLU(inplace=True)
    (2): GINConv(nn=Sequential(
    (0): Linear(in_features=8, out_features=64, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=64, out_features=8, bias=True)
  ))
    (3): ReLU(inplace=True)
    (4): GINConv(nn=Sequential(
    (0): Linear(in_features=8, out_features=64, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=64, out_features=8, bias=True)
  ))
    (5): ReLU(inplace=True)
    (6): GINConv(nn=Sequential(
    (0): Linear(in_features=8, out_features=64, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=64, out_features=8, bias=True)
  ))
    (7): ReLU(inplace=True)
    (8): GINConv(nn=Sequential(
    (0): Linear(in_features=8, out_features=64, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=64, out_features=8, bias=True)
  ))
    (9): ReLU(inplace=True)
    (10): GINConv(nn=Sequential(
    (0): Line

In [92]:
loader = DataLoader(
    dataset,
    batch_size=20000,
    shuffle=False,
    follow_batch=["x_a", "x_b"],
)

In [108]:
with torch.no_grad():
    gnn_output, _, _ = prover._run_gnn_and_attention(next(iter(loader)).to(device))
gnn_output.shape

torch.Size([2, 10000, 11, 8])

In [109]:
pooled_gnn_output = gnn_output.max(dim=2).values
pooled_gnn_output.shape

torch.Size([2, 10000, 8])

In [110]:
all_embeddings = torch.cat((pooled_gnn_output[0], pooled_gnn_output[1]), dim=0)
all_embeddings.shape

torch.Size([20000, 8])

In [111]:
U, S, V = torch.pca_lowrank(all_embeddings)
pca_proj = torch.matmul(all_embeddings, V[:, :2])
pca_proj.shape

torch.Size([20000, 2])

In [112]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=pca_proj[:, 0].detach().numpy(),
        y=pca_proj[:, 1].detach().numpy(),
        mode="markers",
        marker=dict(color="blue"),
    )
)

fig.update_layout(
    title="PCA Projection of Encoded Graphs",
    xaxis_title="PCA Component 1",
    yaxis_title="PCA Component 2",
)

fig.show()

# Training decider directly on encoded graph similarity

In [113]:
decider = GraphIsomorphismSoloProver(params, device).decider

In [114]:
output_close = (
    torch.isclose(pooled_gnn_output[0], pooled_gnn_output[1]).all(dim=-1).long()
)
output_close.shape

torch.Size([10000])

In [115]:
optimizer = Adam(decider.parameters(), lr=LEARNING_RATE)
for epoch in tqdm(range(1000)):
    optimizer.zero_grad()
    pred = decider(gnn_output)
    loss = F.cross_entropy(pred, output_close)
    loss.backward()
    optimizer.step()

with torch.no_grad():
    pred = decider(gnn_output)
    accuracy = (pred.argmax(dim=1) == output_close).float().mean().item()
print(f"Accuracy: {accuracy:.4%}")

100%|██████████| 1000/1000 [00:12<00:00, 83.23it/s]

Accuracy: 58.7300%
